In [1]:
import pandas as pd
import numpy as np


In [2]:
data=pd.read_csv("titanic.csv")

In [3]:
data.head(n=10)

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
5,6,0,3,"Moran, Mr. James",male,NaN,0,0,330877,8.4583,NaN,Q
6,7,0,1,"McCarthy, Mr. Timothy J",male,54.0,0,0,17463,51.8625,E46,S
7,8,0,3,"Palsson, Master. Gosta Leonard",male,2.0,3,1,349909,21.0750,NaN,S
8,9,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.0,0,2,347742,11.1333,NaN,S
9,10,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.0,1,0,237736,30.0708,NaN,C


In [4]:
not_useful_columns=["PassengerId","Name","Embarked","Ticket","Cabin"]

In [5]:
data_clean=data.drop(columns=not_useful_columns)

In [6]:
print(data_clean.Age.mean())

29.69911764705882


In [7]:
data_clean =data_clean.fillna(data_clean.Age.mean())

In [8]:
data_clean.head(n=10)

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,male,22.000000,1,0,7.2500
1,1,1,female,38.000000,1,0,71.2833
2,1,3,female,26.000000,0,0,7.9250
3,1,1,female,35.000000,1,0,53.1000
4,0,3,male,35.000000,0,0,8.0500
5,0,3,male,29.699118,0,0,8.4583
6,0,1,male,54.000000,0,0,51.8625
7,0,3,male,2.000000,3,1,21.0750
8,1,3,female,27.000000,0,2,11.1333
9,1,2,female,14.000000,1,0,30.0708


In [9]:
from sklearn.preprocessing import LabelEncoder

In [10]:
le=LabelEncoder()

In [11]:
data_clean["Sex"]= le.fit_transform(data_clean["Sex"])

In [12]:
data_clean.head()

,Survived,Pclass,Sex,Age,SibSp,Parch,Fare
0,0,3,1,22.0,1,0,7.2500
1,1,1,0,38.0,1,0,71.2833
2,1,3,0,26.0,0,0,7.9250
3,1,1,0,35.0,1,0,53.1000
4,0,3,1,35.0,0,0,8.0500


In [13]:
input_colmns=["Pclass","Sex","Age","Parch","Fare"]
output_colmns=["Survived"]

In [14]:
X = data_clean[input_colmns]
Y = data_clean[output_colmns]


In [15]:
print(X.shape,Y.shape)
print(X.head())
print(data_clean.head())

(891, 5) (891, 1)
   Pclass  Sex   Age  Parch     Fare
0       3    1  22.0      0   7.2500
1       1    0  38.0      0  71.2833
2       3    0  26.0      0   7.9250
3       1    0  35.0      0  53.1000
4       3    1  35.0      0   8.0500
   Survived  Pclass  Sex   Age  SibSp  Parch     Fare
0         0       3    1  22.0      1      0   7.2500
1         1       1    0  38.0      1      0  71.2833
2         1       3    0  26.0      0      0   7.9250
3         1       1    0  35.0      1      0  53.1000
4         0       3    1  35.0      0      0   8.0500


In [15]:
def entropy(column):
    counts=np.unique(column,return_counts=True)
    #print(counts)
    N= float(column.shape[0])
    en=0.0
    for ix in counts[1]:
        p=ix/N
        #print(ix)
        en += (-1.0*p*np.log2(p))
    return en

In [17]:
column=np.array([1,1,1,0,0,0,1,1])
entropy(column)

0.954434002924965

In [16]:
def divide_data(x_data,fkey,fval):
    
    x_left=pd.DataFrame([],columns=x_data.columns)
    x_right=pd.DataFrame([],columns=x_data.columns)
    
    for ix in range(x_data.shape[0]):
        
        val=x_data[fkey].loc[ix]
        
        if val>fval:
            x_right=x_right.append(x_data.loc[ix])
            
        else:
            x_left=x_left.append(x_data.loc[ix])
            
    return x_right,x_left

In [17]:
def info_gain(x_data,fkey,fval):
    
    right,left=divide_data(x_data,fkey,fval)
    
    l=float(left.shape[0]/x_data.shape[0])
    r=float(right.shape[0]/x_data.shape[0])
    
    if left.shape[0]==0 or right.shape[0]==0:
        return -100
    
    i_g=entropy(x_data.Survived)-l*entropy(left.Survived)-r*entropy(right.Survived)
    
    return i_g
    

In [18]:
info_gain(data_clean,'Sex',0.5)

0.2176601066606143

In [19]:
for ix in X.columns:
    print(ix)
    print(info_gain(data_clean,ix,data_clean[ix].mean()))

Pclass
0.07579362743608165
Sex
0.2176601066606143
Age
0.0008836151229467681
Parch
0.015380754493137666
Fare
0.04214069283899541


In [20]:
class decision_tree:
    def __init__(self,depth=0,max_depth=5):
        self.left=None
        self.right=None
        self.depth=depth
        self.max_depth=max_depth
        self.fkey=None
        self.fval=None
        self.target=None
        
    def train(self,Xtrain):
        features=['Pclass', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare']
        i_g=[]
        
        for ix in features:
            ig=info_gain(Xtrain,ix,Xtrain[ix].mean())
            i_g.append(ig)
            
        
        selected= features[np.argmax(i_g)]  
        self.fkey=selected
        self.fval=Xtrain[selected].mean()
        
        left_data,right_data=divide_data(Xtrain,self.fkey,self.fval)
        left_data=left_data.reset_index(drop=True)
        right_data=right_data.reset_index(drop=True)
        
        if left_data.shape[0]==0 or right_data.shape[0]==0:
            if(Xtrain.Survived.mean()>0.5):
                self.target = 1
            else:
                self.target = 0
            return
        
        if(self.depth>=self.max_depth):
            
            if(Xtrain.Survived.mean()>0.5):
                self.target = 1
            else:
                self.target = 0
            return
        
        self.left = decision_tree(depth=self.depth+1,max_depth=self.max_depth)
        self.left.train(left_data)
            
        
        self.right = decision_tree(depth=self.depth+1,max_depth=self.max_depth)
        self.right.train(right_data)
        
        if(Xtrain.Survived.mean()>0.5):
            self.target = 1
        else:
            self.target = 0
        return
    
    
    def predict(self,test):
        if test[self.fkey]>self.fval:
            
            if self.right is None:
                return self.target
            return self.right.predict(test)
        
        else:
            if self.left is None:
                return self.target
            return self.left.predict(test)              

In [21]:
split=int(0.7*data_clean.shape[0])

In [22]:
Xtrain=data_clean[:split]
Xtest=data_clean[split:]

In [23]:
dt=decision_tree()
dt.train(Xtrain)

In [24]:
Xtest=Xtest.reset_index(drop=True)

In [27]:
y_pred=[]
for ix in range(Xtest.shape[0]):
    pr=dt.predict(Xtest.loc[ix])
    y_pred.append(pr)

In [30]:
Y_pred=1-np.array(y_pred)
Y_pred

array([0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1,
       0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0,
       1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1,
       0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1,
       1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 1,
       0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 0,
       1, 1, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1,
       0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1,
       1, 1, 0, 0])

In [31]:
test_data=pd.read_csv('./titanictest.csv')

In [32]:
test_data.shape

(300, 13)

In [33]:
test_data.head()

,pclass,name,sex,age,sibsp,parch,ticket,fare,cabin,embarked,boat,body,home.dest
0,1.0,"Flynn, Mr. John Irwin (""Irving"")",male,36.0,0.0,0.0,PC 17474,26.3875,E25,S,5,NaN,"Brooklyn, NY"
1,3.0,"Sage, Miss. Constance Gladys",female,NaN,8.0,2.0,CA. 2343,69.5500,NaN,S,NaN,NaN,NaN
2,1.0,"Rood, Mr. Hugh Roscoe",male,NaN,0.0,0.0,113767,50.0000,A32,S,NaN,NaN,"Seattle, WA"
3,2.0,"Gillespie, Mr. William Henry",male,34.0,0.0,0.0,12233,13.0000,NaN,S,NaN,NaN,"Vancouver, BC"
4,2.0,"Collander, Mr. Erik Gustaf",male,28.0,0.0,0.0,248740,13.0000,NaN,S,NaN,NaN,"Helsinki, Finland Ashtabula, Ohio"


In [36]:
non_useful_columns=['name','ticket','embarked','cabin','home.dest','sibsp','boat','body']

In [37]:
test_clean=test_data.drop(columns=non_useful_columns)

In [39]:
test_clean.head()

,pclass,sex,age,parch,fare
0,1.0,male,36.0,0.0,26.3875
1,3.0,female,NaN,2.0,69.5500
2,1.0,male,NaN,0.0,50.0000
3,2.0,male,34.0,0.0,13.0000
4,2.0,male,28.0,0.0,13.0000


In [46]:
test_clean =test_clean.fillna(test_clean.age.mean())

In [47]:
test_clean['sex']=le.fit_transform(test_clean['sex'])

In [48]:
test_clean.head()

,pclass,sex,age,parch,fare
0,1.0,1,36.000000,0.0,26.3875
1,3.0,0,29.699118,2.0,69.5500
2,1.0,1,29.699118,0.0,50.0000
3,2.0,1,34.000000,0.0,13.0000
4,2.0,1,28.000000,0.0,13.0000


In [50]:
test_clean.shape

(300, 5)